In [11]:
import json
import re

from redbaron import RedBaron

from openfisca_parsers import contexts, rbnodes, visitors
from openfisca_parsers.ofnodes import is_ofnode
from openfisca_parsers.json_graph import asg_to_json_graph, json_graph_to_asg, deep_diff


## Parse the file to python object

In [3]:
source_file_path = '/data/projects/openfisca/openfisca-france/openfisca_france/model/prelevements_obligatoires/isf.py'
variable_names='isf_imm_bati'
on_parse_error='show'

with open(source_file_path) as source_file:
    source_code = source_file.read()
    
red = RedBaron(source_code)
context = contexts.create()
variable_class_rbnodes = rbnodes.find_all_variable_classes(red, names=variable_names)

for variable_class_rbnode in variable_class_rbnodes:
    variable_name = variable_class_rbnode.name
    visitors.visit_rbnode(variable_class_rbnode, context)

In [4]:
variable_ofnodes = context['variable_by_name'].values()
pyvariable_ofnodes = context['pyvariables']
ofnodes = variable_ofnodes

In [6]:
print json.dumps(ofnodes, indent=4)

[
    {
        "default_value": 0, 
        "input_period": {
            "type": "Period"
        }, 
        "is_period_size_independent": false, 
        "docstring": "Immeubles b\u00e2tis", 
        "label": "isf_imm_bati", 
        "value_type": "float", 
        "output_period": {
            "operator": "this_year", 
            "operand": {
                "type": "Period"
            }, 
            "type": "PeriodOperation"
        }, 
        "formula": {
            "operator": "sum", 
            "type": "ArithmeticOperation", 
            "operands": [
                {
                    "operator": "product", 
                    "type": "ArithmeticOperation", 
                    "operands": [
                        {
                            "operator": "sum", 
                            "type": "ArithmeticOperation", 
                            "operands": [
                                {
                                    "type": "Constant", 
           

## Intepreter

In [64]:
class Instant():
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

class Period():
    def __init__(self, period_type, year, month=None):
        assert period_type in ['year', 'month']
        
        self.period_type = period_type
        self.year = year
        self.month = month
        
    def start(self):
        if self.period_type == 'year':
            return Instant(self.year, 1, 1)
        
        if self.period_type == 'month':
            return Instant(self.year, self.month, 1)
        
    def this_year(self):
        if self.period_type == 'year':
            return self
        
        if self.period_type == 'month':
            return Period('year', self.year)
        
def get_parameter_at_instant(path, instant):
    return 0.3

In [83]:
def evaluate(node, period_values):
    assert is_ofnode(node), node

    nodetype = node['type']
    
    if nodetype == 'Constant':
        return node['value']
    
    if nodetype == 'Variable':
        return evaluate(node['formula'], period_values)
    
    if nodetype == 'ArithmeticOperation':
        operator = node['operator']
        
        if operator == 'negate':
            operand = node['operand']
            return -evaluate(operand, period_values)
        
        if operator in ['sum', 'product']:
            operands_results = [evaluate(op, period_values) for op in node['operands']]
            
            if operator == 'sum':
                return sum(operands_results)
            
            if operator == 'product':
                return reduce(lambda x, y: x * y, operands_results, 1)
        
        raise ValueError('Unknown ArithmeticOperation operator : ' + operator)
        
    if nodetype == 'Period':
        return period_values[id(node)]
        
    if nodetype == 'ParameterAtInstant':
        parameter_path = node['parameter']['path']
        instant = evaluate(node['instant'], period_values)
        return get_parameter_at_instant(parameter_path, instant)
    
    if nodetype == 'PeriodOperation':
        operator = node['operator']
        operand_result = evaluate(node['operand'], period_values)
        
        if operator == 'start':
            return operand_result.start()
        
        if operator == 'this_year':
            return operand_result.this_year()
        
        raise ValueError('Unknown PeriodOperation operator : ' + operator)
        
    if nodetype == 'ValueForPeriod':
        period_result = evaluate(node['period'], period_values)
        variable = node['variable']
            
        if '_stub' in variable and variable['_stub']:
            return 0.65

        variable_period = variable['input_period']
        period_values[id(variable_period)] = period_result
        
        return evaluate(variable, period_values)
    
    raise ValueError('Unknown nodetype : ' + nodetype)

## Basic test

In [84]:
root_node = ofnodes[0]
root_period = root_node['input_period']
period_values = {id(root_period): Period('year', 2015)}

In [85]:
evaluate(root_node, period_values)

1.105